In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
cf.go_offline()
from datetime import datetime
import calendar

In [ ]:
holidays = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')

In [ ]:
train.head()

In [ ]:
store1 = transactions[transactions['store_nbr']==1]

In [ ]:
stor1train = train[train['store_nbr']==1]

In [ ]:
store1['date'] = pd.to_datetime(store1['date'])

In [ ]:
holidays['date'] = pd.to_datetime(holidays['date'])

In [ ]:
store1 = pd.merge(store1,holidays,how='left', on='date')

In [ ]:
holidaytype=['type']
store1= pd.get_dummies(store1,columns=holidaytype, drop_first=True)
locale = ['locale']
store1= pd.get_dummies(store1,columns=locale, drop_first=True)
store1.drop(['description','locale_name','transferred'], axis=1, inplace=True)

In [ ]:
store1['weekday'] = store1['date'].dt.day_name()

In [ ]:
store1

In [ ]:
store1.drop('store_nbr', axis=1, inplace=True)

In [ ]:
fig=plt.figure(figsize=(8,8))
sns.boxplot(x='weekday', y='transactions', data=store1)

In [ ]:
weekday=['weekday']
store1 = pd.get_dummies(store1,columns=weekday, drop_first=True)

In [ ]:
store1['month'] = store1['date'].dt.month_name()

In [ ]:
month=['month']
store1 = pd.get_dummies(store1,columns=month, drop_first=True)

In [ ]:
store1['quitoday'] = (store1['date'].dt.month==12)&(store1['date'].dt.day==6) 
store1['quitoday'] *=1
store1['quitoday'].where(store1['weekday_Sunday'] ==0, other=0, inplace=True)
store1['quitoday'].where(store1['weekday_Saturday'] ==0, other=0, inplace=True)

In [ ]:
store1[store1['quitoday']==1]

In [ ]:
store1['christmas'] = (store1['date'].dt.month==12)&(store1['date'].dt.day==24)
store1['christmas'] *=1
store1['christmas'].where(store1['weekday_Sunday'] ==0, 0, inplace=True)

In [ ]:
#store1['mayday'] = (store1['date'].dt.month==5)&(store1['date'].dt.day==1)
#store1['mayday'] *=1
#store1['mayday'].where(store1['weekday_Sunday'] ==0, 0, inplace=True)

In [ ]:
store1['xmas lead'] =(store1['date'].dt.month==12)&((store1['date'].dt.day>= 21) & (store1['date'].dt.day<=24))
store1['xmas lead'] *=1
store1['xmas lead'].where(store1['weekday_Sunday'] ==0, 0, inplace=True)

In [ ]:
store1['locale_National'].where(store1['weekday_Sunday'] ==0, other=0, inplace=True )
store1['type_Holiday'].where(store1['weekday_Sunday'] ==0, other=0, inplace=True )
#store1['type_Holiday'].where(store1['quitoday']==0, other=0,inplace=True)
store1['type_Transfer'].where(store1['weekday_Sunday'] ==0, other=0, inplace=True )

In [ ]:
#store1[store1['date']=='2016-01-04']
store1.drop(1103, inplace=True) ##Dropping this as an outlier, major earthquake occured on this day

In [ ]:
store1[store1['date'].dt.month ==12].head(10)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
X=store1.drop(['date','transactions'],axis=1)
y=store1['transactions']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
lm=LinearRegression()
lm.fit(X_train,y_train)
predictions = lm.predict(X_test)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
store1['predictions'] = lm.predict(store1.drop(['transactions','date'],axis=1))
store1.iplot(kind='scatter', x='date',y='transactions', secondary_y='predictions')

In [ ]:
store1[store1['date']=='2013-12-06']

In [ ]:
store1['error'] = store1['predictions'] - store1['transactions']
store1.iplot(kind='scatter', x='date', y='error')
store1['error'].iplot(kind='histogram',bins=100)

In [ ]:
store1[abs(store1['error']) >1000]

In [ ]:
errordf = store1[store1['predictions']<0]
errordf[['date','weekday_Sunday','type_Event','type_Holiday','type_Transfer','type_Work Day','locale_National','locale_Regional','transactions','predictions','error']]

In [ ]:
store1[store1['quitoday']==1]

In [ ]:
coeff = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'] )
coeff

In [ ]:
store1.drop(['predictions','error'], axis=1, inplace=True)